# OpenGridMap Notebook -- Installation and Prework

### 1. Install PostGis 

__Mac OSX__: Find installation from link http://postgis.net/install/ ( installed version should be 2.3+)  
__Ubuntu__:  _sudo apt-get install postgresql-contrib postgis_  
To allow remote connection (Here only explain the setting for Ubuntu os):  
    1. Add line   host all your_user_name 0.0.0.0/0 md5   in /etc/postgresql/9.5/main/pg_hba.conf
    2. Set  listen_addresses = '*'  in /etc/postgresql/9.5/main/postgresql.conf
    3. Make sure default listening port 5432 is allowed by your os

### 2. Download map data

The testing data used by this project is __oberbayern-latest.osm.pbf__,  
which is downloaded from link http://download.geofabrik.de/europe/germany/bayern/oberbayern-latest.osm.pbf  
For example, one can use command:  
_wget http://download.geofabrik.de/europe/germany/bayern/oberbayern-latest.osm.pbf _


### 3. Import map data

To import the .osm.pbf data, we need a tool called Osm2pgsql,  
__Mac OSX__:   *brew install osm2pgsql*  
__Ubuntu__:    *wget install osm2pgsql*  
See link *http://wiki.openstreetmap.org/wiki/Osm2pgsql#Special_.27tags.27* for detailed installation. 


After we install it,execute following commands to your PostGis  
<code>
    createdb gis 
	psql -d gis -c 'CREATE EXTENSION postgis; CREATE EXTENSION hstore;'
	osm2pgsql --create --database gis oberbayern-latest.osm.pbf
    **use --slim can slow down the import process and save RAM.
</code>
(More information in link https://github.com/openstreetmap/osm2pgsql)

### 4. Import table for transformers

The .sql file can be found under DataInstallation and use following command to import it:<br>
<code>	psql gis < DataInstallation/student_transformers.sql </code>

### 5. Necessary Tables to create

In [3]:
'''
# Table to store vornoi diagram 
CREATE TABLE vornoi (
    vid bigint NOT NULL,
    tid bigint,
    way geometry(Polygon,900913),
    tway geometry(Point,900913)
);

CREATE SEQUENCE vornoi_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;

ALTER SEQUENCE vornoi_id_seq OWNED BY vornoi.vid;
ALTER TABLE ONLY vornoi ALTER COLUMN vid SET DEFAULT nextval('vornoi_id_seq'::regclass);


# Table to store mapping between houses and vornoi diagram
CREATE TABLE vornoi_map (
    mid bigint NOT NULL,
    oid bigint,
    vid bigint,
    way geometry(Point,900913),
    istransformer BOOLEAN
);

CREATE SEQUENCE vornoi_map_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;

ALTER SEQUENCE vornoi_map_id_seq OWNED BY vornoi_map.mid;
ALTER TABLE ONLY vornoi_map ALTER COLUMN mid SET DEFAULT nextval('vornoi_map_id_seq'::regclass);


# Table to store graph
CREATE TABLE graph (
    gid int NOT NULL,
    vid bigint,
    type VARCHAR(30),
    way geometry(MultiLineString,900913)
);

CREATE SEQUENCE graph_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;

ALTER SEQUENCE graph_id_seq OWNED BY graph.gid;
ALTER TABLE ONLY graph ALTER COLUMN gid SET DEFAULT nextval('graph_id_seq'::regclass);


CREATE TABLE approx_node (
    nid int NOT NULL,
    way geometry(Point,4326),
    ntype int
);

CREATE SEQUENCE approx_node_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;
ALTER SEQUENCE approx_node_id_seq OWNED BY approx_node.nid;
ALTER TABLE ONLY approx_node ALTER COLUMN nid SET DEFAULT nextval('approx_node_id_seq'::regclass);
'''